### Review part 1 [10 Min]:

What is a histogram and what does it show?

### Review part 2 [20 Min]:

Look at the picture for the data about coffee beans:

[Coffee Beans](https://github.com/pointOfive/STA130_F23/blob/main/Tutorial/im/3/coffee_plots.JPG)

Now talk to you group and discribe the data usign what you have leanred in class.

Note: At the end the TA can go throught the main observations so everyone can get the review

### Normal Distributions [20 min]

- [Normal Distribution](https://https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.scribbr.com%2Fstatistics%2Fnormal-distribution%2F&psig=AOvVaw0xQDOX6qolwHo-eWPSKUrz&ust=1715987721207000&source=images&cd=vfe&opi=89978449&ved=0CBIQjRxqFwoTCPCK7sWmk4YDFQAAAAAdAAAAABAY)

- [Left/Right Skew](https://https://github.com/pointOfive/STA130_F23/blob/main/Tutorial/im/3/skew.JPG)

### More About Histiograms [20 min]:

- [Skew/ Modality](https://https://github.com/pointOfive/STA130_F23/blob/main/Tutorial/im/3/skew_modality.png)

- [Skew in Box Plots](https://)




### Small Activity [15 Min]:

Look at the picture of the distrubutions and match them with the categories bellow:

[Question Link](https://github.com/pointOfive/STA130_F23/blob/main/Tutorial/im/3/Quiz2-Question2.png)

1. Variables have similar means and similar variances
2. Variables have similar means but quite different variances
3. Variables have similar variances but quite different means
4. Variables have quite different means and quite different variances


### Activity [15 minutes]:

Watch this video, you will be disgussing it with your groups afterwards

[link text](https://youtu.be/Xz0x-8-cgaQ?si=vWdRwKclNxASv1qL)


### Discussion: [20 min]

Answer the follwoing questions with the people arond you.

- In one sentece what is the main perpose of bootstrapping?
- What are the four steps of bootstrapping?
- The video talks about hypothesis testing a little bit. What does it mean to do hypothesis testing in the context of the video?
